# RO System Simulation with MCAS Property Package

This enhanced notebook template uses the MCAS (Multi-Component Aqueous Solution) property package
for more accurate ion speciation modeling and scaling prediction.

In [ ]:
# Parameters cell - will be replaced by papermill
project_root = "/path/to/project"  # Will be replaced by papermill
configuration = {}
feed_salinity_ppm = 5000
feed_temperature_c = 25.0
membrane_type = "brackish"
membrane_properties = None
optimize_pumps = True  # Changed default to True to match simulate_ro.py
# Ion composition in mg/L (optional, if not provided will use typical composition)
feed_ion_composition = None
# Initialization strategy: "sequential", "block_triangular", "custom_guess", "relaxation"
initialization_strategy = "sequential"

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from pyomo.environ import *
from pyomo.network import Arc
import sys
import os
import platform

# Add parent directory to path for utils imports using project_root parameter
sys.path.insert(0, project_root)

# Ensure PyNumero DLL can be found on Windows
if platform.system() == "Windows":
    pyomo_lib_path = os.path.join(os.path.expanduser("~"), "AppData", "Local", "Pyomo", "lib")
    if os.path.exists(pyomo_lib_path):
        # Add to PATH for DLL loading  
        if 'PATH' in os.environ:
            if pyomo_lib_path not in os.environ['PATH']:
                os.environ['PATH'] = f"{pyomo_lib_path};{os.environ['PATH']}"
        else:
            os.environ['PATH'] = pyomo_lib_path
        print(f"Added Pyomo lib path to PATH: {pyomo_lib_path}")
    
    # Also add to Python's DLL search path
    if hasattr(os, 'add_dll_directory'):
        try:
            os.add_dll_directory(pyomo_lib_path)
            print(f"Added DLL directory: {pyomo_lib_path}")
        except Exception as e:
            print(f"Could not add DLL directory: {e}")

# Verify AmplInterface is available
try:
    from pyomo.contrib.pynumero.asl import AmplInterface
    print(f"AmplInterface available: {AmplInterface.available()}")
except Exception as e:
    print(f"Error checking AmplInterface: {e}")

# Import our MCAS builder
from utils.mcas_builder import (
    build_mcas_property_configuration,
    check_electroneutrality,
    get_total_dissolved_solids,
    calculate_ionic_strength
)

# Import our elegant initialization utilities
from utils.ro_initialization import (
    calculate_required_pressure,
    initialize_pump_with_pressure,
    initialize_ro_unit_elegant,
    initialize_multistage_ro_elegant
)

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType
)
from watertap.unit_models.pressure_changer import Pump
from watertap.property_models.multicomp_aq_sol_prop_pack import MCASParameterBlock

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import calculate_scaling_factors
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.initialization import propagate_state
from idaes.models.unit_models import Feed, Product

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(level=logging.INFO)

# Results storage
results = {}

## Define Feed Ion Composition

In [ ]:
# Define typical ion compositions for different water types
TYPICAL_COMPOSITIONS = {
    "brackish": {
        "Na+": 1200,
        "Ca2+": 120,
        "Mg2+": 60,
        "K+": 20,
        "Cl-": 2100,
        "SO4-2": 200,
        "HCO3-": 150,
        "SiO3-2": 10
    },
    "seawater": {
        "Na+": 10800,
        "Ca2+": 420,
        "Mg2+": 1300,
        "K+": 400,
        "Sr2+": 8,
        "Cl-": 19400,
        "SO4-2": 2700,
        "HCO3-": 140,
        "Br-": 70,
        "F-": 1.3
    }
}

# Use provided composition or typical based on salinity
if feed_ion_composition is None:
    # Scale typical composition to match target salinity
    typical = TYPICAL_COMPOSITIONS[membrane_type]
    typical_tds = sum(typical.values())
    scale_factor = feed_salinity_ppm / typical_tds
    
    feed_ion_composition = {
        ion: conc * scale_factor 
        for ion, conc in typical.items()
    }
else:
    # Parse JSON string if provided
    import json
    if isinstance(feed_ion_composition, str):
        feed_ion_composition = json.loads(feed_ion_composition)

# Check and report composition
print(f"Feed water composition (mg/L):")
for ion, conc in sorted(feed_ion_composition.items()):
    print(f"  {ion:8s}: {conc:8.1f}")

actual_tds = get_total_dissolved_solids(feed_ion_composition)
print(f"\nTotal TDS: {actual_tds:.0f} mg/L")

# Check electroneutrality
is_neutral, imbalance = check_electroneutrality(feed_ion_composition)
if not is_neutral:
    print(f"\nWarning: Charge imbalance of {imbalance:.1%}")

# Calculate ionic strength
ionic_strength = calculate_ionic_strength(feed_ion_composition)
print(f"Ionic strength: {ionic_strength:.3f} mol/L")

## Build MCAS Property Configuration

In [ ]:
# Build MCAS property configuration
mcas_config = build_mcas_property_configuration(
    feed_composition=feed_ion_composition,
    include_scaling_ions=True,
    include_ph_species=True
)

print("MCAS Configuration created with components:")
print(f"  Solute list: {', '.join(mcas_config['solute_list'])}")
print(f"  Activity model: {mcas_config['activity_coefficient_model']}")
print(f"  Scaling ions tracked: {len(mcas_config.get('scaling_ions', {}))} types")

## Build WaterTAP Model with MCAS

In [ ]:
def build_ro_model_mcas(config_data, mcas_config, feed_temperature_c, membrane_type):
    """
    Build WaterTAP RO model with MCAS property package using SKK transport model.
    """
    # Create concrete model
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Import MaterialFlowBasis and TransportModel
    from watertap.property_models.multicomp_aq_sol_prop_pack import MaterialFlowBasis
    from watertap.core.membrane_channel_base import TransportModel
    
    # Filter MCAS config to only include parameters MCASParameterBlock accepts
    mcas_params = {
        'solute_list': mcas_config['solute_list'],
        'mw_data': mcas_config['mw_data'],
        'material_flow_basis': MaterialFlowBasis.mass,  # Use mass flow for RO compatibility
    }
    
    # Add optional parameters if they exist
    optional_params = [
        'stokes_radius_data', 'diffusivity_data', 'molar_volume_data',
        'elec_mobility_data', 'trans_num_data', 'equiv_conductivity_phase_data',
        'charge', 'ignore_neutral_charge', 'activity_coefficient_model',
        'density_calculation', 'diffus_calculation', 'elec_mobility_calculation',
        'trans_num_calculation'
    ]
    
    for param in optional_params:
        if param in mcas_config:
            mcas_params[param] = mcas_config[param]
    
    # Create MCAS property package with filtered parameters
    m.fs.properties = MCASParameterBlock(**mcas_params)
    
    # Feed conditions
    feed_flow_m3_s = config_data['feed_flow_m3h'] / 3600  # Convert to m³/s
    
    # Create feed block
    m.fs.feed = Feed(property_package=m.fs.properties)
    
    # Build stages - use proper attribute assignment for pumps and RO units
    n_stages = config_data['stage_count']
    
    # First, create all units (without setting parameters)
    for i in range(1, n_stages + 1):
        # Create pump for stage using setattr to ensure proper parent assignment
        setattr(m.fs, f"pump{i}", Pump(property_package=m.fs.properties))
        
        # Create RO stage with SKK transport model
        setattr(m.fs, f"ro_stage{i}", ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            concentration_polarization_type=ConcentrationPolarizationType.calculated,
            mass_transfer_coefficient=MassTransferCoefficient.calculated,
            pressure_change_type=PressureChangeType.fixed_per_stage,
            transport_model=TransportModel.SKK  # Use SKK model
        ))
        
        # Create permeate product for each stage
        setattr(m.fs, f"stage_product{i}", Product(property_package=m.fs.properties))
    
    # Build connectivity
    # Feed to first pump
    m.fs.feed_to_pump1 = Arc(
        source=m.fs.feed.outlet,
        destination=m.fs.pump1.inlet
    )
    
    # First pump to first RO
    m.fs.pump1_to_ro1 = Arc(
        source=m.fs.pump1.outlet,
        destination=m.fs.ro_stage1.inlet
    )
    
    # Permeate from first RO to product
    m.fs.ro1_perm_to_prod = Arc(
        source=m.fs.ro_stage1.permeate,
        destination=m.fs.stage_product1.inlet
    )
    
    # Connect stages if multiple
    if n_stages > 1:
        for i in range(1, n_stages):
            # Concentrate of stage i to pump i+1
            arc_name = f"ro{i}_to_pump{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"ro_stage{i}").retentate,
                destination=getattr(m.fs, f"pump{i+1}").inlet
            ))
            
            # Pump i+1 to RO i+1
            arc_name = f"pump{i+1}_to_ro{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"pump{i+1}").outlet,
                destination=getattr(m.fs, f"ro_stage{i+1}").inlet
            ))
            
            # Permeate to product
            arc_name = f"ro{i+1}_perm_to_prod{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"ro_stage{i+1}").permeate,
                destination=getattr(m.fs, f"stage_product{i+1}").inlet
            ))
    
    # Final concentrate product
    m.fs.concentrate_product = Product(property_package=m.fs.properties)
    final_stage = n_stages
    m.fs.final_conc_arc = Arc(
        source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
        destination=m.fs.concentrate_product.inlet
    )
    
    # Apply arcs to expand the network
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    # NOW set membrane properties after model structure is built
    for i in range(1, n_stages + 1):
        stage_data = config_data['stages'][i-1]
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Set reflection coefficient for SKK model (typical values 0.9-0.98)
        ro.reflect_coeff.fix(0.95)  # High rejection for salts
        
        # Membrane properties based on type
        if membrane_type == "seawater":
            ro.A_comp.fix(1.5e-12)  # m/s/Pa
            # Set B values only for solutes (not H2O)
            for comp in mcas_config['solute_list']:
                # Different rejection for different ions
                if comp in ['Na_+', 'Cl_-']:
                    ro.B_comp[0, comp].fix(1.0e-8)  # m/s
                elif comp in ['Ca_2+', 'Mg_2+', 'SO4_2-']:
                    ro.B_comp[0, comp].fix(5.0e-9)  # m/s - higher rejection
                else:
                    ro.B_comp[0, comp].fix(8.0e-9)  # m/s
        else:  # brackish
            ro.A_comp.fix(4.2e-12)  # m/s/Pa
            for comp in mcas_config['solute_list']:
                if comp in ['Na_+', 'Cl_-']:
                    ro.B_comp[0, comp].fix(3.5e-8)  # m/s
                elif comp in ['Ca_2+', 'Mg_2+', 'SO4_2-']:
                    ro.B_comp[0, comp].fix(1.5e-8)  # m/s
                else:
                    ro.B_comp[0, comp].fix(2.5e-8)  # m/s
        
        # Set fixed pressure drop (0.5 bar typical)
        ro.deltaP.fix(-0.5e5)  # Pa
        
        # FIX permeate pressure when has_pressure_change=True (required for DOF=0)
        ro.permeate.pressure[0].fix(101325)  # 1 atm
        
        # Channel geometry for concentration polarization calculations
        ro.feed_side.channel_height.fix(0.001)  # 1 mm
        ro.feed_side.spacer_porosity.fix(0.85)
        
        # Set membrane area and width
        # WaterTAP requires both area and width to be fixed (area = width * length constraint)
        # Handle both 'membrane_area_m2' and 'area_m2' field names for compatibility
        if 'membrane_area_m2' in stage_data:
            required_area = stage_data['membrane_area_m2']
        elif 'area_m2' in stage_data:
            required_area = stage_data['area_m2']
        else:
            # Calculate from vessel count if not provided
            n_vessels = stage_data.get('n_vessels', stage_data.get('vessel_count', 1))
            vessel_area = 37.16 * 7  # Default: 37.16 m² per element * 7 elements per vessel
            required_area = n_vessels * vessel_area
        
        ro.area.fix(required_area)
        
        # Fix width to a reasonable value based on stage configuration
        # This represents the aggregate width across all vessels in the stage
        n_vessels = stage_data.get('n_vessels', stage_data.get('vessel_count', 1))
        aggregate_width = n_vessels * 5.0  # Assume 5m effective width per vessel group
        ro.width.fix(aggregate_width)
        
        # Length will be calculated from area = width * length constraint
        # No need to fix length - it will be determined by the model
    
    # Set feed conditions
    feed_state = m.fs.feed.outlet
    
    # Temperature and pressure
    feed_state.temperature.fix(273.15 + feed_temperature_c)
    feed_state.pressure.fix(101325)  # 1 atm
    
    # Component flows based on ion composition (now using mass flow)
    ion_composition_mg_l = mcas_config['ion_composition_mg_l']
    total_ion_flow_kg_s = 0
    
    # Set ion flows (mass basis)
    for comp in mcas_config['solute_list']:
        # Ion flow
        conc_mg_l = ion_composition_mg_l[comp]
        ion_flow_kg_s = conc_mg_l * feed_flow_m3_s / 1000  # mg/L * m³/s / 1000 = kg/s
        feed_state.flow_mass_phase_comp[0, 'Liq', comp].fix(ion_flow_kg_s)
        total_ion_flow_kg_s += ion_flow_kg_s
    
    # Water flow (mass basis)
    water_flow_kg_s = feed_flow_m3_s * 1000 - total_ion_flow_kg_s  # Assume density ~1000 kg/m³
    feed_state.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(water_flow_kg_s)
    
    return m

## Initialize Model with Robust Strategy

In [ ]:
def build_ro_model_mcas(config_data, mcas_config, feed_temperature_c, membrane_type):
    """
    Build WaterTAP RO model with MCAS property package using SD transport model.
    
    Note: Using SD (Solution-Diffusion) model instead of SKK for compatibility
    with multi-component systems. SKK model currently requires all B_comp values
    to be identical due to single scalar alpha constraint.
    """
    # Create concrete model
    m = ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)
    
    # Import MaterialFlowBasis and TransportModel
    from watertap.property_models.multicomp_aq_sol_prop_pack import MaterialFlowBasis
    from watertap.core.membrane_channel_base import TransportModel
    
    # Filter MCAS config to only include parameters MCASParameterBlock accepts
    mcas_params = {
        'solute_list': mcas_config['solute_list'],
        'mw_data': mcas_config['mw_data'],
        'material_flow_basis': MaterialFlowBasis.mass,  # Use mass flow for RO compatibility
    }
    
    # Add optional parameters if they exist
    optional_params = [
        'stokes_radius_data', 'diffusivity_data', 'molar_volume_data',
        'elec_mobility_data', 'trans_num_data', 'equiv_conductivity_phase_data',
        'charge', 'ignore_neutral_charge', 'activity_coefficient_model',
        'density_calculation', 'diffus_calculation', 'elec_mobility_calculation',
        'trans_num_calculation'
    ]
    
    for param in optional_params:
        if param in mcas_config:
            mcas_params[param] = mcas_config[param]
    
    # Create MCAS property package with filtered parameters
    m.fs.properties = MCASParameterBlock(**mcas_params)
    
    # Feed conditions
    feed_flow_m3_s = config_data['feed_flow_m3h'] / 3600  # Convert to m³/s
    
    # Create feed block
    m.fs.feed = Feed(property_package=m.fs.properties)
    
    # Build stages - use proper attribute assignment for pumps and RO units
    n_stages = config_data['stage_count']
    
    # First, create all units (without setting parameters)
    for i in range(1, n_stages + 1):
        # Create pump for stage using setattr to ensure proper parent assignment
        setattr(m.fs, f"pump{i}", Pump(property_package=m.fs.properties))
        
        # Create RO stage with SD transport model for multi-component compatibility
        setattr(m.fs, f"ro_stage{i}", ReverseOsmosis0D(
            property_package=m.fs.properties,
            has_pressure_change=True,
            concentration_polarization_type=ConcentrationPolarizationType.calculated,
            mass_transfer_coefficient=MassTransferCoefficient.calculated,
            pressure_change_type=PressureChangeType.fixed_per_stage,
            transport_model=TransportModel.SD  # Use SD model for multi-component
        ))
        
        # Create permeate product for each stage
        setattr(m.fs, f"stage_product{i}", Product(property_package=m.fs.properties))
    
    # Build connectivity
    # Feed to first pump
    m.fs.feed_to_pump1 = Arc(
        source=m.fs.feed.outlet,
        destination=m.fs.pump1.inlet
    )
    
    # First pump to first RO
    m.fs.pump1_to_ro1 = Arc(
        source=m.fs.pump1.outlet,
        destination=m.fs.ro_stage1.inlet
    )
    
    # Permeate from first RO to product
    m.fs.ro1_perm_to_prod = Arc(
        source=m.fs.ro_stage1.permeate,
        destination=m.fs.stage_product1.inlet
    )
    
    # Connect stages if multiple
    if n_stages > 1:
        for i in range(1, n_stages):
            # Concentrate of stage i to pump i+1
            arc_name = f"ro{i}_to_pump{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"ro_stage{i}").retentate,
                destination=getattr(m.fs, f"pump{i+1}").inlet
            ))
            
            # Pump i+1 to RO i+1
            arc_name = f"pump{i+1}_to_ro{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"pump{i+1}").outlet,
                destination=getattr(m.fs, f"ro_stage{i+1}").inlet
            ))
            
            # Permeate to product
            arc_name = f"ro{i+1}_perm_to_prod{i+1}"
            setattr(m.fs, arc_name, Arc(
                source=getattr(m.fs, f"ro_stage{i+1}").permeate,
                destination=getattr(m.fs, f"stage_product{i+1}").inlet
            ))
    
    # Final concentrate product
    m.fs.concentrate_product = Product(property_package=m.fs.properties)
    final_stage = n_stages
    m.fs.final_conc_arc = Arc(
        source=getattr(m.fs, f"ro_stage{final_stage}").retentate,
        destination=m.fs.concentrate_product.inlet
    )
    
    # Apply arcs to expand the network
    TransformationFactory("network.expand_arcs").apply_to(m)
    
    # NOW set membrane properties after model structure is built
    for i in range(1, n_stages + 1):
        stage_data = config_data['stages'][i-1]
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Note: SD model doesn't use reflection coefficient
        # Different B_comp values for different ions are supported
        
        # Membrane properties based on type
        if membrane_type == "seawater":
            ro.A_comp.fix(1.5e-12)  # m/s/Pa
            # Set B values only for solutes (not H2O)
            for comp in mcas_config['solute_list']:
                # Different rejection for different ions
                if comp in ['Na_+', 'Cl_-']:
                    ro.B_comp[0, comp].fix(1.0e-8)  # m/s
                elif comp in ['Ca_2+', 'Mg_2+', 'SO4_2-']:
                    ro.B_comp[0, comp].fix(5.0e-9)  # m/s - higher rejection
                else:
                    ro.B_comp[0, comp].fix(8.0e-9)  # m/s
        else:  # brackish
            ro.A_comp.fix(4.2e-12)  # m/s/Pa
            for comp in mcas_config['solute_list']:
                if comp in ['Na_+', 'Cl_-']:
                    ro.B_comp[0, comp].fix(3.5e-8)  # m/s
                elif comp in ['Ca_2+', 'Mg_2+', 'SO4_2-']:
                    ro.B_comp[0, comp].fix(1.5e-8)  # m/s
                else:
                    ro.B_comp[0, comp].fix(2.5e-8)  # m/s
        
        # Set fixed pressure drop (0.5 bar typical)
        ro.deltaP.fix(-0.5e5)  # Pa
        
        # FIX permeate pressure when has_pressure_change=True (required for DOF=0)
        ro.permeate.pressure[0].fix(101325)  # 1 atm
        
        # Channel geometry for concentration polarization calculations
        ro.feed_side.channel_height.fix(0.001)  # 1 mm
        ro.feed_side.spacer_porosity.fix(0.85)
        
        # Set membrane area and width
        # WaterTAP requires both area and width to be fixed (area = width * length constraint)
        # Handle both 'membrane_area_m2' and 'area_m2' field names for compatibility
        if 'membrane_area_m2' in stage_data:
            required_area = stage_data['membrane_area_m2']
        elif 'area_m2' in stage_data:
            required_area = stage_data['area_m2']
        else:
            # Calculate from vessel count if not provided
            n_vessels = stage_data.get('n_vessels', stage_data.get('vessel_count', 1))
            vessel_area = 37.16 * 7  # Default: 37.16 m² per element * 7 elements per vessel
            required_area = n_vessels * vessel_area
        
        ro.area.fix(required_area)
        
        # Fix width to a reasonable value based on stage configuration
        # This represents the aggregate width across all vessels in the stage
        n_vessels = stage_data.get('n_vessels', stage_data.get('vessel_count', 1))
        aggregate_width = n_vessels * 5.0  # Assume 5m effective width per vessel group
        ro.width.fix(aggregate_width)
        
        # Length will be calculated from area = width * length constraint
        # No need to fix length - it will be determined by the model
    
    # Set feed conditions
    feed_state = m.fs.feed.outlet
    
    # Temperature and pressure
    feed_state.temperature.fix(273.15 + feed_temperature_c)
    feed_state.pressure.fix(101325)  # 1 atm
    
    # Component flows based on ion composition (now using mass flow)
    ion_composition_mg_l = mcas_config['ion_composition_mg_l']
    total_ion_flow_kg_s = 0
    
    # Set ion flows (mass basis)
    for comp in mcas_config['solute_list']:
        # Ion flow
        conc_mg_l = ion_composition_mg_l[comp]
        ion_flow_kg_s = conc_mg_l * feed_flow_m3_s / 1000  # mg/L * m³/s / 1000 = kg/s
        feed_state.flow_mass_phase_comp[0, 'Liq', comp].fix(ion_flow_kg_s)
        total_ion_flow_kg_s += ion_flow_kg_s
    
    # Water flow (mass basis)
    water_flow_kg_s = feed_flow_m3_s * 1000 - total_ion_flow_kg_s  # Assume density ~1000 kg/m³
    feed_state.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(water_flow_kg_s)
    
    return m

In [ ]:
def initialize_and_solve_mcas(m, config_data, optimize_pumps=False):
    """
    Initialize and solve the RO model with MCAS using elegant initialization.
    
    When optimize_pumps=False: Just initialize and solve with fixed pumps
    When optimize_pumps=True: Unfix pumps and add recovery constraints to match targets exactly
    """
    # Get solver
    solver = get_solver()
    
    # Check initial degrees of freedom
    print(f"Initial degrees of freedom: {degrees_of_freedom(m)}")
    
    # Use elegant initialization approach (pumps are fixed during this step)
    try:
        initialize_multistage_ro_elegant(m, config_data, verbose=True)
        print("\nInitialization successful using elegant approach!")
    except Exception as e:
        print(f"\nInitialization error: {str(e)}")
        # Fall back to sequential initialization
        print("Falling back to sequential initialization...")
        initialize_model_sequential(m, config_data)
    
    if not optimize_pumps:
        # Simple solve with fixed pumps (no recovery constraints)
        print("\nSolving with fixed pump pressures...")
        results = solver.solve(m, tee=True, options={'linear_solver': 'ma27'})
        
        if results.solver.termination_condition == TerminationCondition.optimal:
            print("\nSolution found with fixed pumps!")
            # Report actual recoveries achieved
            print("\nActual recoveries achieved:")
            for i in range(1, config_data['stage_count'] + 1):
                ro = getattr(m.fs, f"ro_stage{i}")
                # Water recovery
                h2o_recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
                print(f"  Stage {i}: Water recovery = {h2o_recovery:.1%}")
        
        return results
    
    # If optimize_pumps=True, proceed with optimization
    print("\nOptimization requested - unfixing pump pressures...")
    
    # First solve with fixed pumps to verify feasibility
    print("\nVerifying initial solution...")
    results = solver.solve(m, tee=False, options={'linear_solver': 'ma27'})
    if results.solver.termination_condition != TerminationCondition.optimal:
        print(f"Initial solve failed: {results.solver.termination_condition}")
        raise RuntimeError("Initial solve failed after initialization")
    
    # Now unfix pump pressures for optimization
    for i in range(1, config_data['stage_count'] + 1):
        pump = getattr(m.fs, f"pump{i}")
        current_pressure = value(pump.outlet.pressure[0])
        
        # Unfix and set bounds
        pump.outlet.pressure[0].unfix()
        pump.outlet.pressure[0].setlb(5e5)   # 5 bar minimum
        pump.outlet.pressure[0].setub(80e5)  # 80 bar maximum
        
        print(f"  Stage {i}: Unfixed at {current_pressure/1e5:.1f} bar, bounds: [5, 80] bar")
    
    # Add recovery constraints to match configuration targets
    print("\nAdding recovery constraints...")
    from pyomo.environ import Constraint, Var
    
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        target_recovery = stage_data.get('stage_recovery', 0.5)
        
        # Add constraint for water recovery
        setattr(m.fs, f"recovery_constraint_stage{i}",
            Constraint(
                expr=ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'] == target_recovery
            )
        )
        print(f"  Stage {i}: Target water recovery = {target_recovery:.1%}")
    
    # Note: We do NOT add explicit minimum driving pressure constraints for SD model
    # The SD model's flux equation automatically ensures sufficient driving pressure:
    # flux = A × ρ × ((P_feed - P_perm) - (π_feed - π_perm))
    # If pressure is insufficient, flux will be zero/negative and recovery targets cannot be met
    print("\nNote: SD model flux equation automatically handles driving pressure requirements")
    
    # Check DOF after adding constraints
    print(f"\nDegrees of freedom after recovery constraints: {degrees_of_freedom(m)}")
    
    # Solve with recovery constraints and unfixed pumps
    print("\nSolving model with recovery constraints...")
    results = solver.solve(m, tee=True, options={'linear_solver': 'ma27'})
    
    if results.solver.termination_condition == TerminationCondition.optimal:
        # Verify recoveries match targets
        print("\nVerifying recovery targets:")
        max_error = 0
        for i in range(1, config_data['stage_count'] + 1):
            ro = getattr(m.fs, f"ro_stage{i}")
            actual_recovery = value(ro.recovery_mass_phase_comp[0, 'Liq', 'H2O'])
            target_recovery = config_data['stages'][i-1].get('stage_recovery', 0.5)
            error = abs(actual_recovery - target_recovery)
            max_error = max(max_error, error)
            print(f"  Stage {i}: Target = {target_recovery:.4f}, Actual = {actual_recovery:.4f}, Error = {error:.2e}")
        
        if max_error > 1e-4:
            print(f"\nWARNING: Maximum recovery error ({max_error:.2e}) exceeds tolerance!")
        else:
            print(f"\nSUCCESS: All recoveries match targets within tolerance!")
        
        # Report optimized conditions with SD model details
        print("\nOptimized conditions (SD model):")
        for i in range(1, config_data['stage_count'] + 1):
            pump = getattr(m.fs, f"pump{i}")
            ro = getattr(m.fs, f"ro_stage{i}")
            
            # Get pressures
            feed_pressure = value(pump.outlet.pressure[0]) / 1e5
            perm_pressure = value(ro.permeate.pressure[0]) / 1e5
            
            # Get osmotic pressures at the membrane interface (where it matters)
            feed_osm = value(ro.feed_side.properties_interface[0, 0].pressure_osm_phase['Liq']) / 1e5
            perm_osm = value(ro.permeate_side[0, 0].pressure_osm_phase['Liq']) / 1e5
            
            # Calculate net driving pressure for SD model
            net_driving = (feed_pressure - perm_pressure) - (feed_osm - perm_osm)
            
            # Get water flux (determined by recovery constraint)
            water_flux = value(ro.flux_mass_phase_comp[0, 0, 'Liq', 'H2O']) / 1000  # kg/m²/s to m³/m²/s
            water_flux_lmh = water_flux * 3.6e6  # Convert to L/m²/h
            
            print(f"\n  Stage {i}:")
            print(f"    Feed pressure: {feed_pressure:.2f} bar")
            print(f"    Feed osmotic pressure (interface): {feed_osm:.2f} bar")
            print(f"    Permeate osmotic pressure: {perm_osm:.2f} bar")
            print(f"    Net driving pressure: {net_driving:.2f} bar")
            print(f"    Water flux: {water_flux_lmh:.1f} LMH")
            
            # Show concentrate TDS to verify pressure scaling
            conc_flow = value(ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O'])
            conc_tds = 0
            for comp in m.fs.properties.solute_set:
                conc_tds += value(ro.retentate.flow_mass_phase_comp[0, 'Liq', comp])
            if conc_flow > 0:
                conc_tds_ppm = (conc_tds / (conc_flow + conc_tds)) * 1e6
                print(f"    Concentrate TDS: {conc_tds_ppm:.0f} ppm")
            
            # Show ion-specific permeate quality
            print(f"    Ion rejection:")
            for comp in m.fs.properties.solute_set:
                feed_conc = value(ro.feed_side.properties_interface[0, 0].conc_mass_phase_comp['Liq', comp])
                perm_conc = value(ro.permeate_side[0, 0].conc_mass_phase_comp['Liq', comp])
                if feed_conc > 0:
                    rejection = 1 - (perm_conc / feed_conc)
                    print(f"      {comp:8s}: {rejection:.1%}")
        
        # Calculate total power
        total_power_kw = sum(
            value(getattr(m.fs, f"pump{i}").work_mechanical[0]) / 1000
            for i in range(1, config_data['stage_count'] + 1)
        )
        print(f"\nTotal power consumption: {total_power_kw:.1f} kW")
    
    return results


def initialize_model_sequential(m, config_data):
    """
    Fallback sequential initialization (original method).
    """
    # Initialize feed
    m.fs.feed.initialize()
    
    # Initialize stages sequentially
    for i in range(1, config_data['stage_count'] + 1):
        print(f"\nInitializing Stage {i}...")
        
        # Initialize pump
        pump = getattr(m.fs, f"pump{i}")
        
        # Set outlet pressure based on stage and expected osmotic pressure
        if i == 1:
            pump.outlet.pressure.fix(15e5)  # 15 bar
        elif i == 2:
            pump.outlet.pressure.fix(25e5)  # 25 bar (higher due to concentration)
        else:
            pump.outlet.pressure.fix(35e5)  # 35 bar
        
        pump.efficiency_pump.fix(0.8)
        
        # Propagate state from previous unit
        if i == 1:
            propagate_state(arc=m.fs.feed_to_pump1)
        else:
            arc_name = f"ro{i-1}_to_pump{i}"
            propagate_state(arc=getattr(m.fs, arc_name))
        
        pump.initialize()
        
        # Initialize RO
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Propagate state from pump
        if i == 1:
            propagate_state(arc=m.fs.pump1_to_ro1)
        else:
            arc_name = f"pump{i}_to_ro{i}"
            propagate_state(arc=getattr(m.fs, arc_name))
        
        # Initialize RO
        ro.initialize()
        
        # Initialize stage product
        if i == 1:
            propagate_state(arc=m.fs.ro1_perm_to_prod)
        else:
            arc_name = f"ro{i}_perm_to_prod{i}"
            propagate_state(arc=getattr(m.fs, arc_name))
        
        getattr(m.fs, f"stage_product{i}").initialize()
    
    # Initialize final concentrate product
    propagate_state(arc=m.fs.final_conc_arc)
    m.fs.concentrate_product.initialize()
    
    print("\nSequential initialization complete.")

In [ ]:
def initialize_with_block_triangularization(m, config_data):
    """
    Initialize using block triangularization for strongly connected components.
    """
    from pyomo.environ import Block
    from idaes.core.initialization import BlockTriangularizationInitializer
    
    print("Initializing with block triangularization...")
    
    # Create initializer
    initializer = BlockTriangularizationInitializer()
    
    # Initialize each unit in sequence
    units = [m.fs.feed]
    for i in range(1, config_data['stage_count'] + 1):
        units.append(getattr(m.fs, f"pump{i}"))
        units.append(getattr(m.fs, f"ro_stage{i}"))
        units.append(getattr(m.fs, f"stage_product{i}"))
    units.append(m.fs.concentrate_product)
    
    for unit in units:
        try:
            initializer.initialize(unit)
        except:
            # Fall back to default initialization
            unit.initialize()


def initialize_with_custom_guess(m, config_data):
    """
    Initialize with custom initial guesses based on typical values.
    """
    print("Setting custom initial values...")
    
    # Typical pressure progression
    stage_pressures = {
        1: 15e5,   # 15 bar
        2: 25e5,   # 25 bar  
        3: 35e5    # 35 bar
    }
    
    # Set pump pressures
    for i in range(1, config_data['stage_count'] + 1):
        pump = getattr(m.fs, f"pump{i}")
        if i in stage_pressures:
            pump.outlet.pressure.set_value(stage_pressures[i])
    
    # Set RO recoveries based on configuration (as initial guesses only)
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        target_recovery = stage_data.get('stage_recovery', 0.5)
        
        # Set recovery for each component (just as initial values, not fixed)
        for comp in m.fs.properties.component_list:
            if comp == "H2O":
                ro.recovery_mass_phase_comp[0, 'Liq', comp].set_value(target_recovery)
            else:
                # Assume 98% rejection for ions
                ro.recovery_mass_phase_comp[0, 'Liq', comp].set_value(0.02)
    
    # Set approximate flows
    feed_flow = config_data['feed_flow_m3h'] / 3600  # m³/s
    
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Approximate permeate and concentrate flows
        if i == 1:
            inlet_flow = feed_flow
        else:
            # Previous stage concentrate
            inlet_flow = feed_flow * (1 - 0.5 * (i-1))
        
        stage_recovery = config_data['stages'][i-1].get('stage_recovery', 0.5)
        perm_flow = inlet_flow * stage_recovery
        conc_flow = inlet_flow - perm_flow
        
        # Set approximate values (mass basis)
        ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O'].set_value(perm_flow * 1000)
        ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O'].set_value(conc_flow * 1000)


def initialize_with_relaxation(m, config_data):
    """
    Initialize with constraint relaxation for difficult problems.
    """
    from pyomo.core.plugins.transform.relax_integrality import RelaxIntegrality
    
    print("Initializing with constraint relaxation...")
    
    # First, set custom guesses
    initialize_with_custom_guess(m, config_data)
    
    # Initialize units
    units = [m.fs.feed]
    for i in range(1, config_data['stage_count'] + 1):
        units.append(getattr(m.fs, f"pump{i}"))
        units.append(getattr(m.fs, f"ro_stage{i}"))
    
    for unit in units:
        unit.initialize()


def initialize_model_advanced(m, config_data, strategy="sequential"):
    """
    Initialize model using selected strategy.
    
    Args:
        m: Pyomo model
        config_data: Configuration data
        strategy: Initialization strategy
            - "sequential": Default sequential initialization
            - "block_triangular": Block triangularization
            - "custom_guess": Custom initial values
            - "relaxation": Constraint relaxation
    """
    print(f"\nInitializing model using {strategy} strategy...")
    
    if strategy == "sequential":
        initialize_model_robust(m, config_data)
    elif strategy == "block_triangular":
        initialize_with_block_triangularization(m, config_data)
    elif strategy == "custom_guess":
        initialize_with_custom_guess(m, config_data)
        initialize_model_robust(m, config_data)
    elif strategy == "relaxation":
        initialize_with_relaxation(m, config_data)
    else:
        print(f"Unknown strategy {strategy}, using sequential")
        initialize_model_robust(m, config_data)
    
    # Verify initialization
    print("\nChecking initialization...")
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        perm_flow = value(sum(
            ro.permeate.flow_mass_phase_comp[0, 'Liq', comp]
            for comp in m.fs.properties.component_list
        )) / 1000 * 3600  # m³/h
        
        print(f"  Stage {i} permeate flow: {perm_flow:.1f} m³/h")

## Advanced Initialization Strategies

In [ ]:
def solve_with_scaling(m, optimize_pumps=False):
    """
    Solve model with appropriate scaling.
    """
    # Apply scaling factors
    m.fs.properties.set_default_scaling('flow_mol_phase_comp', 1e2, index=('Liq', 'H2O'))
    m.fs.properties.set_default_scaling('flow_mol_phase_comp', 1e4, index=('Liq', 'Na+'))
    m.fs.properties.set_default_scaling('flow_mol_phase_comp', 1e4, index=('Liq', 'Cl-'))
    
    # Calculate scaling factors
    calculate_scaling_factors(m)
    
    # Get solver
    solver = get_solver()
    solver.options['max_iter'] = 200
    
    # Check DOF
    print(f"\nDegrees of freedom: {degrees_of_freedom(m)}")
    
    # Solve
    print("\nSolving model...")
    results = solver.solve(m, tee=True)
    
    if optimize_pumps and results.solver.termination_condition == TerminationCondition.optimal:
        print("\nOptimizing pump pressures...")
        # TODO: Implement pump optimization
        # Unfix pump pressures and add objective function
        pass
    
    return results

In [ ]:
def add_lcow_objective(m, config_data):
    """
    Add LCOW minimization objective to the model.
    """
    from pyomo.environ import Objective, minimize
    from utils.config import get_config
    
    # Get economic parameters
    electricity_cost = get_config('energy.electricity_cost_usd_kwh', 0.07)
    membrane_cost_m2 = get_config('capital.membrane_cost_usd_m2', 30.0)
    power_cost_kw = get_config('capital.power_equipment_cost_usd_kw', 1000.0)
    indirect_factor = get_config('capital.indirect_cost_factor', 2.5)
    membrane_life = get_config('operating.membrane_lifetime_years', 7)
    plant_life = get_config('financial.plant_lifetime_years', 20)
    discount_rate = get_config('financial.discount_rate', 0.08)
    plant_availability = get_config('operating.plant_availability', 0.9)
    maintenance_frac = get_config('operating.maintenance_fraction', 0.02)
    
    # Calculate capital recovery factor
    crf = (discount_rate * (1 + discount_rate)**plant_life) / \
          ((1 + discount_rate)**plant_life - 1)
    
    # Total membrane area (fixed)
    total_membrane_area = config_data['total_membrane_area_m2']
    
    # Total power (variable - depends on pump pressures)
    m.fs.total_power_kw = Var(
        initialize=100,
        bounds=(10, 1000),
        doc="Total power consumption in kW"
    )
    
    # Constraint: Total power = sum of pump powers
    m.fs.total_power_constraint = Constraint(
        expr=m.fs.total_power_kw == sum(
            getattr(m.fs, f"pump{i}").work_mechanical[0] / 1000  # W to kW
            for i in range(1, config_data['stage_count'] + 1)
        )
    )
    
    # Annual production (m³/year)
    total_perm_flow_m3_s = sum(
        sum(
            getattr(m.fs, f"stage_product{i}").inlet.flow_mass_phase_comp[0, 'Liq', comp]
            for comp in m.fs.properties.component_list
        ) / 1000  # kg/s to m³/s
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    annual_production = total_perm_flow_m3_s * 3600 * 24 * 365 * plant_availability
    
    # Capital costs
    membrane_capital = total_membrane_area * membrane_cost_m2
    power_capital = m.fs.total_power_kw * 1.2 * power_cost_kw  # 20% margin
    direct_capital = membrane_capital + power_capital
    total_capital = direct_capital * indirect_factor
    
    # Annual costs
    annual_capital = total_capital * crf
    annual_energy = annual_production * m.fs.total_power_kw / (total_perm_flow_m3_s * 3600) * electricity_cost
    annual_membrane = membrane_capital / membrane_life
    annual_maintenance = total_capital * maintenance_frac
    
    total_annual_cost = annual_capital + annual_energy + annual_membrane + annual_maintenance
    
    # LCOW objective
    m.fs.lcow = Var(
        initialize=1.0,
        bounds=(0.1, 10),
        doc="Levelized cost of water ($/m³)"
    )
    
    m.fs.lcow_constraint = Constraint(
        expr=m.fs.lcow == total_annual_cost / annual_production
    )
    
    # Minimize LCOW
    m.fs.objective = Objective(expr=m.fs.lcow, sense=minimize)
    
    return m


def optimize_pump_pressures(m, config_data):
    """
    Optimize pump outlet pressures to minimize LCOW.
    """
    # Unfix pump outlet pressures
    for i in range(1, config_data['stage_count'] + 1):
        pump = getattr(m.fs, f"pump{i}")
        pump.outlet.pressure.unfix()
        pump.outlet.pressure.setlb(5e5)   # 5 bar minimum
        pump.outlet.pressure.setub(80e5)  # 80 bar maximum
    
    # Add pressure constraints
    # Each stage must have higher pressure than the previous
    if config_data['stage_count'] > 1:
        for i in range(1, config_data['stage_count']):
            pump_current = getattr(m.fs, f"pump{i}")
            pump_next = getattr(m.fs, f"pump{i+1}")
            m.fs.add_component(
                f"pressure_sequence_{i}",
                Constraint(
                    expr=pump_next.outlet.pressure[0] >= 
                         pump_current.outlet.pressure[0] + 1e5  # At least 1 bar higher
                )
            )
    
    # Ensure sufficient pressure for desired recovery
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        stage_data = config_data['stages'][i-1]
        target_recovery = stage_data.get('stage_recovery', 0.5)
        
        # Approximate pressure requirement (simplified)
        # P_feed > π_feed + ΔP + P_perm
        # Where π is osmotic pressure
        # For MCAS with ions, need to calculate total solute contribution
        total_solute_frac = sum(
            ro.inlet.flow_mass_phase_comp[0, 'Liq', comp] / 
            sum(ro.inlet.flow_mass_phase_comp[0, 'Liq', c] for c in m.fs.properties.component_list)
            for comp in m.fs.properties.solute_set
        )
        
        # Van't Hoff approximation for osmotic pressure
        osmotic_pressure = 0.7 * total_solute_frac * 1e6  # Approximate, Pa
        
        m.fs.add_component(
            f"min_pressure_stage_{i}",
            Constraint(
                expr=ro.inlet.pressure[0] >= osmotic_pressure * (1 + target_recovery) + 2e5
            )
        )
    
    # Add the LCOW objective
    m = add_lcow_objective(m, config_data)
    
    # Re-solve with optimization
    solver = get_solver()
    solver.options['max_iter'] = 500
    solver.options['tol'] = 1e-6
    
    print("\\nOptimizing pump pressures to minimize LCOW...")
    results = solver.solve(m, tee=True)
    
    if results.solver.termination_condition == TerminationCondition.optimal:
        print(f"\\nOptimization successful!")
        print(f"Minimized LCOW: ${value(m.fs.lcow):.3f}/m³")
        print(f"\\nOptimized pump pressures:")
        for i in range(1, config_data['stage_count'] + 1):
            pump = getattr(m.fs, f"pump{i}")
            print(f"  Stage {i}: {value(pump.outlet.pressure[0])/1e5:.1f} bar")
    else:
        print(f"\\nOptimization failed: {results.solver.termination_condition}")
    
    return results

def solve_with_scaling(m, config_data, optimize_pumps=False):
    """
    Solve model with appropriate scaling.
    """
    # Apply scaling factors
    m.fs.properties.set_default_scaling('flow_mol_phase_comp', 1e2, index=('Liq', 'H2O'))
    m.fs.properties.set_default_scaling('flow_mol_phase_comp', 1e4, index=('Liq', 'Na+'))
    m.fs.properties.set_default_scaling('flow_mol_phase_comp', 1e4, index=('Liq', 'Cl-'))
    
    # Calculate scaling factors
    calculate_scaling_factors(m)
    
    # Get solver
    solver = get_solver()
    solver.options['max_iter'] = 200
    
    # Check DOF
    print(f"\nDegrees of freedom: {degrees_of_freedom(m)}")
    
    # Solve
    print("\nSolving model...")
    results = solver.solve(m, tee=True)
    
    if optimize_pumps and results.solver.termination_condition == TerminationCondition.optimal:
        print("\nStarting pump pressure optimization...")
        results = optimize_pump_pressures(m, config_data)
    
    return results

## Scaling Prediction with Reaktoro

In [ ]:
def predict_scaling_potential(m, stage_num):
    """
    Predict scaling potential using concentrate composition.
    """
    # For now, return a simplified scaling assessment
    # In production, this would use a proper scaling prediction module
    
    ro = getattr(m.fs, f"ro_stage{stage_num}")
    conc_state = ro.retentate
    
    # Extract ion concentrations in mg/L
    conc_ions_mg_l = {}
    total_flow_kg_s = 0
    
    for comp in m.fs.properties.component_list:
        if comp != "H2O":
            # Get mass flow directly (already in mass basis)
            comp_flow_kg_s = value(conc_state.flow_mass_phase_comp[0, 'Liq', comp])
            conc_ions_mg_l[comp] = comp_flow_kg_s  # Will convert to mg/L later
        else:
            water_flow_kg_s = value(conc_state.flow_mass_phase_comp[0, 'Liq', 'H2O'])
            total_flow_kg_s += water_flow_kg_s
    
    # Convert to mg/L
    total_flow_m3_s = total_flow_kg_s / 1000  # Approximate density 1000 kg/m³
    if total_flow_m3_s > 0:
        for ion in conc_ions_mg_l:
            # kg/s / (m³/s) * 1e6 mg/kg = mg/L
            conc_ions_mg_l[ion] = conc_ions_mg_l[ion] / total_flow_m3_s * 1e6
    
    # Simple scaling assessment based on common scaling compounds
    scaling_results = {}
    
    # For simple NaCl solution, no scaling risk
    # Add more sophisticated checks if needed for other ions
    scaling_results['NaCl'] = {
        'saturation_index': -1.0,
        'scaling_tendency': 'Low'
    }
    
    # Antiscalant recommendation (simplified)
    antiscalant_rec = {
        'antiscalant_type': 'None required',
        'dosage_ppm': 0,
        'primary_concern': 'None'
    }
    
    return scaling_results, antiscalant_rec

def predict_scaling_potential(m, stage_num):
    """
    Predict scaling potential using concentrate composition.
    """
    # For now, return a simplified scaling assessment
    # In production, this would use a proper scaling prediction module
    
    ro = getattr(m.fs, f"ro_stage{stage_num}")
    conc_state = ro.retentate
    
    # Extract ion concentrations in mg/L
    conc_ions_mg_l = {}
    total_flow_kg_s = 0
    
    for comp in m.fs.properties.component_list:
        if comp != "H2O":
            # Get mass flow directly (already in mass basis)
            comp_flow_kg_s = value(conc_state.flow_mass_phase_comp[0, 'Liq', comp])
            conc_ions_mg_l[comp] = comp_flow_kg_s  # Will convert to mg/L later
        else:
            water_flow_kg_s = value(conc_state.flow_mass_phase_comp[0, 'Liq', 'H2O'])
            total_flow_kg_s += water_flow_kg_s
    
    # Convert to mg/L
    total_flow_m3_s = total_flow_kg_s / 1000  # Approximate density 1000 kg/m³
    if total_flow_m3_s > 0:
        for ion in conc_ions_mg_l:
            # kg/s / (m³/s) * 1e6 mg/kg = mg/L
            conc_ions_mg_l[ion] = conc_ions_mg_l[ion] / total_flow_m3_s * 1e6
    
    # Simple scaling assessment based on common scaling compounds
    scaling_results = {}
    
    # Calcium carbonate scaling (simplified)
    if 'Ca2+' in conc_ions_mg_l and 'HCO3-' in conc_ions_mg_l:
        ca_mol_l = conc_ions_mg_l.get('Ca2+', 0) / 40080  # mg/L to mol/L
        hco3_mol_l = conc_ions_mg_l.get('HCO3-', 0) / 61020  # mg/L to mol/L
        # Very simplified Langelier Saturation Index approximation
        lsi_approx = 0.5 * (ca_mol_l * hco3_mol_l * 1e6) - 1.0
        scaling_results['CaCO3'] = {
            'saturation_index': lsi_approx,
            'scaling_tendency': 'High' if lsi_approx > 0.5 else 'Medium' if lsi_approx > 0 else 'Low'
        }
    
    # Calcium sulfate scaling
    if 'Ca2+' in conc_ions_mg_l and 'SO4-2' in conc_ions_mg_l:
        ca_mg_l = conc_ions_mg_l.get('Ca2+', 0)
        so4_mg_l = conc_ions_mg_l.get('SO4-2', 0)
        # Simple check against typical solubility
        if (ca_mg_l * so4_mg_l) > 500000:  # Simplified threshold
            scaling_results['CaSO4'] = {
                'saturation_index': 1.0,
                'scaling_tendency': 'High'
            }
        else:
            scaling_results['CaSO4'] = {
                'saturation_index': -0.5,
                'scaling_tendency': 'Low'
            }
    
    # Antiscalant recommendation (simplified)
    antiscalant_rec = {
        'antiscalant_type': 'None required',
        'dosage_ppm': 0,
        'primary_concern': 'None'
    }
    
    # Check if any scaling risk exists
    high_risk_scales = [k for k, v in scaling_results.items() 
                        if v.get('scaling_tendency') == 'High']
    
    if high_risk_scales:
        if 'CaCO3' in high_risk_scales:
            antiscalant_rec = {
                'antiscalant_type': 'Phosphonate-based',
                'dosage_ppm': 3.0,
                'primary_concern': 'Calcium carbonate',
                'specific_products': ['FLOCON 260', 'Vitec 3000']
            }
        elif 'CaSO4' in high_risk_scales:
            antiscalant_rec = {
                'antiscalant_type': 'Polyacrylate-based',
                'dosage_ppm': 2.5,
                'primary_concern': 'Calcium sulfate',
                'specific_products': ['FLOCON 100', 'Vitec 2000']
            }
    
    return scaling_results, antiscalant_rec

In [ ]:
def extract_results_mcas(m, config_data):
    """
    Extract results including detailed ion concentrations.
    """
    results = {
        "status": "success",
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {},
        "ion_analysis": {}
    }
    
    # Get feed flow (mass basis)
    feed_vol_flow = sum(
        value(m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', comp])  # kg/s
        for comp in m.fs.properties.component_list
    ) / 1000  # m³/s assuming density ~1000
    
    # Total permeate flow
    total_perm_flow = sum(
        sum(
            value(getattr(m.fs, f"stage_product{i}").inlet.flow_mass_phase_comp[0, 'Liq', comp])
            for comp in m.fs.properties.component_list
        ) / 1000
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    results["performance"]["total_recovery"] = total_perm_flow / feed_vol_flow
    
    # Energy consumption
    total_power_kw = sum(
        value(getattr(m.fs, f"pump{i}").work_mechanical[0]) / 1000
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    results["economics"]["total_power_kw"] = total_power_kw
    results["economics"]["specific_energy_kwh_m3"] = (
        total_power_kw / (total_perm_flow * 3600)
    )
    
    # Stage results with ion details
    for i in range(1, config_data['stage_count'] + 1):
        ro = getattr(m.fs, f"ro_stage{i}")
        
        # Calculate flows and TDS
        perm_tds = 0
        perm_ions = {}
        
        for comp in m.fs.properties.component_list:
            if comp != "H2O":
                perm_kg_s = value(ro.permeate.flow_mass_phase_comp[0, 'Liq', comp])
                perm_mg_s = perm_kg_s * 1e6  # kg/s to mg/s
                perm_flow_m3_s = value(ro.permeate.flow_mass_phase_comp[0, 'Liq', 'H2O']) / 1000
                perm_conc_mg_l = perm_mg_s / (perm_flow_m3_s * 1000) if perm_flow_m3_s > 0 else 0
                perm_ions[comp] = perm_conc_mg_l
                perm_tds += perm_conc_mg_l
        
        # Similar for concentrate
        conc_tds = 0
        conc_ions = {}
        
        for comp in m.fs.properties.component_list:
            if comp != "H2O":
                conc_kg_s = value(ro.retentate.flow_mass_phase_comp[0, 'Liq', comp])
                conc_mg_s = conc_kg_s * 1e6  # kg/s to mg/s
                conc_flow_m3_s = value(ro.retentate.flow_mass_phase_comp[0, 'Liq', 'H2O']) / 1000
                conc_conc_mg_l = conc_mg_s / (conc_flow_m3_s * 1000) if conc_flow_m3_s > 0 else 0
                conc_ions[comp] = conc_conc_mg_l
                conc_tds += conc_conc_mg_l
        
        # Predict scaling for this stage
        scaling_results, antiscalant_rec = predict_scaling_potential(m, i)
        
        stage_result = {
            "stage_number": i,
            "feed_pressure_bar": value(ro.inlet.pressure[0]) / 1e5,
            "permeate_flow_m3h": sum(
                value(ro.permeate.flow_mass_phase_comp[0, 'Liq', comp])
                for comp in m.fs.properties.component_list
            ) / 1000 * 3600,
            "permeate_tds_ppm": perm_tds,
            "permeate_ions_mg_l": perm_ions,
            "concentrate_flow_m3h": sum(
                value(ro.retentate.flow_mass_phase_comp[0, 'Liq', comp])
                for comp in m.fs.properties.component_list
            ) / 1000 * 3600,
            "concentrate_tds_ppm": conc_tds,
            "concentrate_ions_mg_l": conc_ions,
            "pump_power_kw": value(getattr(m.fs, f"pump{i}").work_mechanical[0]) / 1000,
            "scaling_indices": {
                mineral: data["saturation_index"] 
                for mineral, data in scaling_results.items() 
                if "saturation_index" in data
            },
            "scaling_risks": {
                mineral: data["scaling_tendency"] 
                for mineral, data in scaling_results.items() 
                if "scaling_tendency" in data
            },
            "antiscalant_recommendation": antiscalant_rec
        }
        
        results["stage_results"].append(stage_result)
    
    # Overall ion rejection
    overall_rejection = {}
    for comp in m.fs.properties.component_list:
        if comp != "H2O":
            feed_kg_s = value(m.fs.feed.outlet.flow_mass_phase_comp[0, 'Liq', comp])
            perm_kg_s = sum(
                value(getattr(m.fs, f"stage_product{i}").inlet.flow_mass_phase_comp[0, 'Liq', comp])
                for i in range(1, config_data['stage_count'] + 1)
            )
            if feed_kg_s > 0:
                overall_rejection[comp] = 1 - (perm_kg_s / feed_kg_s)
    
    results["ion_analysis"]["overall_rejection"] = overall_rejection
    
    # Mass balance
    results["mass_balance"]["feed_flow_m3h"] = feed_vol_flow * 3600
    results["mass_balance"]["total_permeate_m3h"] = total_perm_flow * 3600
    results["mass_balance"]["final_concentrate_m3h"] = sum(
        value(m.fs.concentrate_product.inlet.flow_mass_phase_comp[0, 'Liq', comp])
        for comp in m.fs.properties.component_list
    ) / 1000 * 3600
    
    balance_error = abs(
        results["mass_balance"]["feed_flow_m3h"] - 
        results["mass_balance"]["total_permeate_m3h"] - 
        results["mass_balance"]["final_concentrate_m3h"]
    )
    results["mass_balance"]["error_m3h"] = balance_error
    results["mass_balance"]["error_percent"] = (
        balance_error / results["mass_balance"]["feed_flow_m3h"] * 100
    )
    
    return results

def extract_results_mcas(m, config_data):
    """
    Extract results including detailed ion concentrations.
    """
    results = {
        "status": "success",
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {},
        "ion_analysis": {}
    }
    
    # Get feed flow
    feed_vol_flow = sum(
        value(m.fs.feed.outlet.flow_mol_phase_comp[0, 'Liq', comp] * 
              m.fs.properties.get_component(comp).mw / 1000)  # kg/s
        for comp in m.fs.properties.component_list
    ) / 1000  # m³/s assuming density ~1000
    
    # Total permeate flow
    total_perm_flow = sum(
        sum(
            value(m.fs.stage_products[i].inlet.flow_mol_phase_comp[0, 'Liq', comp] * 
                  m.fs.properties.get_component(comp).mw / 1000)
            for comp in m.fs.properties.component_list
        ) / 1000
        for i in range(1, config_data['stage_count'] + 1)
    )
    
    results["performance"]["total_recovery"] = total_perm_flow / feed_vol_flow
    
    # Energy consumption
    total_power_kw = sum(
        value(pump.work_mechanical[0]) / 1000
        for pump in m.fs.pumps.values()
    )
    
    results["economics"]["total_power_kw"] = total_power_kw
    results["economics"]["specific_energy_kwh_m3"] = (
        total_power_kw / (total_perm_flow * 3600)
    )
    
    # Stage results with ion details
    for i in range(1, config_data['stage_count'] + 1):
        ro = m.fs.ro_stages[i]
        
        # Calculate flows and TDS
        perm_tds = 0
        perm_ions = {}
        
        for comp in m.fs.properties.component_list:
            if comp != "H2O":
                perm_mol = value(ro.permeate.flow_mol_phase_comp[0, 'Liq', comp])
                perm_mg_s = perm_mol * m.fs.properties.get_component(comp).mw * 1000
                perm_flow_m3_s = value(ro.permeate.flow_mol_phase_comp[0, 'Liq', 'H2O']) * 0.018015 / 1000
                perm_conc_mg_l = perm_mg_s / perm_flow_m3_s if perm_flow_m3_s > 0 else 0
                perm_ions[comp] = perm_conc_mg_l
                perm_tds += perm_conc_mg_l
        
        # Similar for concentrate
        conc_tds = 0
        conc_ions = {}
        
        for comp in m.fs.properties.component_list:
            if comp != "H2O":
                conc_mol = value(ro.retentate.flow_mol_phase_comp[0, 'Liq', comp])
                conc_mg_s = conc_mol * m.fs.properties.get_component(comp).mw * 1000
                conc_flow_m3_s = value(ro.retentate.flow_mol_phase_comp[0, 'Liq', 'H2O']) * 0.018015 / 1000
                conc_conc_mg_l = conc_mg_s / conc_flow_m3_s if conc_flow_m3_s > 0 else 0
                conc_ions[comp] = conc_conc_mg_l
                conc_tds += conc_conc_mg_l
        
        # Predict scaling for this stage
        scaling_results, antiscalant_rec = predict_scaling_potential(m, i)
        
        stage_result = {
            "stage_number": i,
            "feed_pressure_bar": value(ro.inlet.pressure[0]) / 1e5,
            "permeate_flow_m3h": sum(
                value(ro.permeate.flow_mol_phase_comp[0, 'Liq', comp] * 
                      m.fs.properties.get_component(comp).mw / 1000)
                for comp in m.fs.properties.component_list
            ) / 1000 * 3600,
            "permeate_tds_ppm": perm_tds,
            "permeate_ions_mg_l": perm_ions,
            "concentrate_flow_m3h": sum(
                value(ro.retentate.flow_mol_phase_comp[0, 'Liq', comp] * 
                      m.fs.properties.get_component(comp).mw / 1000)
                for comp in m.fs.properties.component_list
            ) / 1000 * 3600,
            "concentrate_tds_ppm": conc_tds,
            "concentrate_ions_mg_l": conc_ions,
            "pump_power_kw": value(m.fs.pumps[i].work_mechanical[0]) / 1000,
            "scaling_indices": {
                mineral: data["saturation_index"] 
                for mineral, data in scaling_results.items() 
                if "saturation_index" in data
            },
            "scaling_risks": {
                mineral: data["scaling_tendency"] 
                for mineral, data in scaling_results.items() 
                if "scaling_tendency" in data
            },
            "antiscalant_recommendation": antiscalant_rec
        }
        
        results["stage_results"].append(stage_result)
    
    # Overall ion rejection
    overall_rejection = {}
    for comp in m.fs.properties.component_list:
        if comp != "H2O":
            feed_mol = value(m.fs.feed.outlet.flow_mol_phase_comp[0, 'Liq', comp])
            perm_mol = sum(
                value(m.fs.stage_products[i].inlet.flow_mol_phase_comp[0, 'Liq', comp])
                for i in range(1, config_data['stage_count'] + 1)
            )
            if feed_mol > 0:
                overall_rejection[comp] = 1 - (perm_mol / feed_mol)
    
    results["ion_analysis"]["overall_rejection"] = overall_rejection
    
    # Mass balance
    results["mass_balance"]["feed_flow_m3h"] = feed_vol_flow * 3600
    results["mass_balance"]["total_permeate_m3h"] = total_perm_flow * 3600
    results["mass_balance"]["final_concentrate_m3h"] = sum(
        value(m.fs.concentrate_product.inlet.flow_mol_phase_comp[0, 'Liq', comp] * 
              m.fs.properties.get_component(comp).mw / 1000)
        for comp in m.fs.properties.component_list
    ) / 1000 * 3600
    
    balance_error = abs(
        results["mass_balance"]["feed_flow_m3h"] - 
        results["mass_balance"]["total_permeate_m3h"] - 
        results["mass_balance"]["final_concentrate_m3h"]
    )
    results["mass_balance"]["error_m3h"] = balance_error
    results["mass_balance"]["error_percent"] = (
        balance_error / results["mass_balance"]["feed_flow_m3h"] * 100
    )
    
    return results

In [ ]:
# Build and solve model
try:
    print("Building RO model with MCAS property package...")
    m = build_ro_model_mcas(configuration, mcas_config, feed_temperature_c, membrane_type)
    
    # Apply scaling
    print("\nApplying scaling factors...")
    m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1, index=('Liq', 'H2O'))
    for comp in mcas_config['solute_list']:
        m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e4, index=('Liq', comp))
    calculate_scaling_factors(m)
    
    print("\nInitializing and solving model...")
    solve_results = initialize_and_solve_mcas(m, configuration, optimize_pumps)
    
    if solve_results.solver.termination_condition == TerminationCondition.optimal:
        print("\nExtracting results...")
        results = extract_results_mcas(m, configuration)
        results["message"] = "Simulation with MCAS completed successfully"
        results["property_package"] = "MCAS"
        results["components_modeled"] = ['H2O'] + mcas_config['solute_list']
        results["initialization_strategy"] = "elegant"
        
        # Add optimization results if performed
        if optimize_pumps and hasattr(m.fs, 'obj'):
            results["optimization"] = {
                "performed": True,
                "minimized_power_kw": value(m.fs.total_power)/1000,
                "optimized_pressures_bar": {
                    f"stage_{i}": value(getattr(m.fs, f"pump{i}").outlet.pressure[0])/1e5
                    for i in range(1, configuration['stage_count'] + 1)
                }
            }
    else:
        results = {
            "status": "error",
            "message": f"Solver failed: {solve_results.solver.termination_condition}",
            "performance": {},
            "economics": {},
            "stage_results": [],
            "mass_balance": {},
            "ion_analysis": {}
        }
        
except Exception as e:
    import traceback
    results = {
        "status": "error",
        "message": f"Simulation error: {str(e)}",
        "traceback": traceback.format_exc(),
        "performance": {},
        "economics": {},
        "stage_results": [],
        "mass_balance": {},
        "ion_analysis": {}
    }

print("\nSimulation complete.")

## Display Results

In [ ]:
# Display results summary
import json
print("\n" + "="*60)
print("SIMULATION RESULTS - MCAS PROPERTY PACKAGE")
print("="*60)

if results.get("status") == "success":
    print(f"\nOverall Performance:")
    print(f"  Total Recovery: {results['performance']['total_recovery']:.1%}")
    print(f"  Specific Energy: {results['economics']['specific_energy_kwh_m3']:.2f} kWh/m³")
    print(f"  Total Power: {results['economics']['total_power_kw']:.1f} kW")
    
    print(f"\nIon Rejection:")
    for ion, rejection in results['ion_analysis']['overall_rejection'].items():
        print(f"  {ion:8s}: {rejection:.1%}")
    
    print(f"\nStage Results:")
    for stage in results['stage_results']:
        print(f"\n  Stage {stage['stage_number']}:")
        print(f"    Pressure: {stage['feed_pressure_bar']:.1f} bar")
        print(f"    Permeate: {stage['permeate_flow_m3h']:.1f} m³/h @ {stage['permeate_tds_ppm']:.0f} ppm")
        print(f"    Concentrate: {stage['concentrate_flow_m3h']:.1f} m³/h @ {stage['concentrate_tds_ppm']:.0f} ppm")
        
        if stage.get('scaling_indices'):
            print(f"    Scaling Risk:")
            for mineral, si in stage['scaling_indices'].items():
                if si > 0:
                    print(f"      {mineral}: SI = {si:.2f} (supersaturated)")

# Full results as JSON
print("\n" + "="*60)
print("FULL RESULTS (JSON):")
print("="*60)
print(json.dumps(results, indent=2, default=str))

In [ ]:
# Display results summary
import json
print("\n" + "="*60)
print("SIMULATION RESULTS - MCAS PROPERTY PACKAGE")
print("="*60)

if results.get("status") == "success":
    print(f"\nOverall Performance:")
    print(f"  Total Recovery: {results['performance']['total_recovery']:.1%}")
    print(f"  Specific Energy: {results['economics']['specific_energy_kwh_m3']:.2f} kWh/m³")
    print(f"  Total Power: {results['economics']['total_power_kw']:.1f} kW")
    
    print(f"\nIon Rejection:")
    for ion, rejection in results['ion_analysis']['overall_rejection'].items():
        print(f"  {ion:8s}: {rejection:.1%}")
    
    print(f"\nStage Results:")
    for stage in results['stage_results']:
        print(f"\n  Stage {stage['stage_number']}:")
        print(f"    Pressure: {stage['feed_pressure_bar']:.1f} bar")
        print(f"    Permeate: {stage['permeate_flow_m3h']:.1f} m³/h @ {stage['permeate_tds_ppm']:.0f} ppm")
        print(f"    Concentrate: {stage['concentrate_flow_m3h']:.1f} m³/h @ {stage['concentrate_tds_ppm']:.0f} ppm")
        
        # Show scaling risks
        if stage.get('scaling_indices'):
            print(f"    Scaling Risk Assessment:")
            for mineral, si in sorted(stage['scaling_indices'].items()):
                if si > -0.5:  # Only show minerals near or above saturation
                    risk = stage['scaling_risks'].get(mineral, '')
                    print(f"      {mineral:12s}: SI = {si:+.2f} - {risk}")
        
        # Show antiscalant recommendation if scaling risk exists
        if stage.get('antiscalant_recommendation'):
            rec = stage['antiscalant_recommendation']
            if rec['antiscalant_type'] != "None required":
                print(f"    Antiscalant Recommendation:")
                print(f"      Primary concern: {rec['primary_concern']}")
                print(f"      Type: {rec['antiscalant_type']}")
                print(f"      Dosage: {rec['dosage_ppm']:.1f} ppm")
                if rec.get('specific_products'):
                    print(f"      Products: {', '.join(rec['specific_products'][:2])}")

# Full results as JSON
print("\n" + "="*60)
print("FULL RESULTS (JSON):")
print("="*60)
print(json.dumps(results, indent=2, default=str))

In [ ]:
# Results cell - tagged for papermill to extract
results